In [1]:

import pandas as pd
import numpy as np
import requests
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import accuracy_score
from datetime import datetime, timedelta

# OpenWeather API Key (Replace with your actual key)
API_KEY = "c5615a8df07bf39f21e624ffe731f157"

# Load dataset
file_path = r"C:\Users\harsh\Downloads\Cyclone.csv"
df = pd.read_csv(file_path)

# Drop unnecessary columns
df = df.drop(columns=["index", "FID", "BTID", "NAME", "BASIN"])

# Encode categorical target variable
le = LabelEncoder()
df["CAT"] = le.fit_transform(df["CAT"])

# Define features and targets
X = df[["LAT", "LONG", "WIND_KTS", "PRESSURE", "YEAR"]]
y_cat = df["CAT"]
y_shape = df["Shape_Leng"]

# Split dataset into training (85%) and testing (15%)
X_train, X_test, y_cat_train, y_cat_test, y_shape_train, y_shape_test = train_test_split(
    X, y_cat, y_shape, test_size=0.15, random_state=42
)

# Scale numerical features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Train RandomForest models
clf = RandomForestClassifier(n_estimators=500, random_state=42, n_jobs=-1)
clf.fit(X_train, y_cat_train)

reg_shape = RandomForestRegressor(n_estimators=500, random_state=42, n_jobs=-1)
reg_shape.fit(X_train, y_shape_train)

# Cyclone category descriptions and intensity levels
category_mapping = {
    "E": ("Extratropical Cyclone", "Nothing to worry"),
    "H1": ("Category 1 Hurricane - Weak hurricane (64–82 knots wind speed)", "Medium intensity"),
    "H2": ("Category 2 Hurricane - Moderate hurricane (83–95 knots wind speed)", "High intensity"),
    "H3": ("Category 3 Hurricane - Strong hurricane (96–112 knots wind speed)", "High intensity"),
    "H4": ("Category 4 Hurricane - Very strong hurricane (113–136 knots wind speed)", "High intensity"),
    "H5": ("Category 5 Hurricane - Extremely dangerous hurricane (137+ knots wind speed)", "High intensity"),
    "L": ("Low-Pressure System - A developing weather system", "Nothing to worry"),
    "SD": ("Subtropical Depression - Hybrid tropical cyclone", "Normal"),
    "SS": ("Subtropical Storm - Hybrid tropical cyclone", "Medium intensity"),
    "TD": ("Tropical Depression - Weak tropical cyclone (≤ 33 knots wind speed)", "Normal"),
    "TS": ("Tropical Storm - Medium-intensity tropical cyclone (34–63 knots wind speed)", "Medium intensity"),
    "W": ("Weak System - Possibly a developing disturbance", "Nothing to worry"),
}

# Function to fetch weather data
def fetch_weather(city):
    url = f"http://api.openweathermap.org/data/2.5/weather?q={city}&appid={API_KEY}&units=metric"
    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()
        lat = data['coord']['lat']
        lon = data['coord']['lon']
        wind_speed_kts = data['wind']['speed'] * 1.944  # Convert m/s to knots
        pressure = data['main']['pressure']
        
        # Extract date from API response
        timestamp = data['dt']
        date = datetime.utcfromtimestamp(timestamp).strftime('%Y-%m-%d')

        return lat, lon, wind_speed_kts, pressure, date
    else:
        print("Error fetching weather data.")
        return None

# Prediction function
def predict_cyclone():
    city = input("Enter city name: ")
    weather_data = fetch_weather(city)

    if weather_data:
        lat, long, wind_kts, pressure, date = weather_data
        year = int(date.split('-')[0])

        print("\n--- Weather Data Fetched ---")
        print(f"City: {city}")
        print(f"Latitude: {lat}")
        print(f"Longitude: {long}")
        print(f"Wind Speed (knots): {wind_kts:.2f}")
        print(f"Pressure: {pressure} hPa")
        print(f"Date: {date}")

        # Prepare input data for today's prediction
        input_data = pd.DataFrame([[lat, long, wind_kts, pressure, year]], 
                                  columns=["LAT", "LONG", "WIND_KTS", "PRESSURE", "YEAR"])
        input_data_scaled = scaler.transform(input_data)

        # Predictions for today
        category_pred = clf.predict(input_data_scaled)
        shape_length_pred = reg_shape.predict(input_data_scaled)
        category_label = le.inverse_transform(category_pred)[0]

        # Get description & intensity alert
        description, intensity_alert = category_mapping.get(category_label, ("Unknown Category", "Unknown Intensity"))

        # Calculate classification accuracy
        cat_accuracy = accuracy_score(y_cat_test, clf.predict(X_test))

        print("\n--- Cyclone Prediction for Today ---")
        print(f"Cyclone Category: {category_label}")
        print(f"Description: {description}")
        print(f"Predicted Shape Length: {shape_length_pred[0]:.2f} km")
        print(f"Model Classification Accuracy: {cat_accuracy:.2%}")
        print(f"⚠ Intensity Alert: {intensity_alert}")

        # Predict for next 10 days
        print("\n--- Cyclone Prediction for Next 10 Days ---")
        for i in range(1, 11):
            future_date = datetime.strptime(date, '%Y-%m-%d') + timedelta(days=i)
            future_year = future_date.year

            input_data_future = pd.DataFrame([[lat, long, wind_kts, pressure, future_year]], 
                                             columns=["LAT", "LONG", "WIND_KTS", "PRESSURE", "YEAR"])
            input_data_scaled_future = scaler.transform(input_data_future)

            category_pred_future = clf.predict(input_data_scaled_future)
            shape_length_pred_future = reg_shape.predict(input_data_scaled_future)
            category_label_future = le.inverse_transform(category_pred_future)[0]

            description_future, intensity_alert_future = category_mapping.get(category_label_future, 
                                                                              ("Unknown Category", "Unknown Intensity"))

            print(f"\nPredicted Cyclone for {future_date.strftime('%Y-%m-%d')}:")
            print(f"Cyclone Category: {category_label_future}")
            print(f"Description: {description_future}")
            print(f"Predicted Shape Length: {shape_length_pred_future[0]:.2f} km")
            print(f"⚠ Intensity Alert: {intensity_alert_future}")

# Run the prediction function

In [2]:
predict_cyclone()

C:\Users\harsh\AppData\Local\Temp\ipykernel_9572\4068659249.py:76: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  date = datetime.utcfromtimestamp(timestamp).strftime('%Y-%m-%d')



--- Weather Data Fetched ---
City: chennai
Latitude: 13.0878
Longitude: 80.2785
Wind Speed (knots): 9.00
Pressure: 1009 hPa
Date: 2025-02-15

--- Cyclone Prediction for Today ---
Cyclone Category: TD
Description: Tropical Depression - Weak tropical cyclone (≤ 33 knots wind speed)
Predicted Shape Length: 1.06 km
Model Classification Accuracy: 97.37%
⚠ Intensity Alert: Normal

--- Cyclone Prediction for Next 10 Days ---

Predicted Cyclone for 2025-02-16:
Cyclone Category: TD
Description: Tropical Depression - Weak tropical cyclone (≤ 33 knots wind speed)
Predicted Shape Length: 1.06 km
⚠ Intensity Alert: Normal

Predicted Cyclone for 2025-02-17:
Cyclone Category: TD
Description: Tropical Depression - Weak tropical cyclone (≤ 33 knots wind speed)
Predicted Shape Length: 1.06 km
⚠ Intensity Alert: Normal

Predicted Cyclone for 2025-02-18:
Cyclone Category: TD
Description: Tropical Depression - Weak tropical cyclone (≤ 33 knots wind speed)
Predicted Shape Length: 1.06 km
⚠ Intensity Alert: